In [1]:
# Standard library imports
import os
from collections import Counter, defaultdict
import random

# Hugging Face datasets for loading corpus
from datasets import load_dataset

# NLTK for natural language processing
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.lm import MLE, Vocabulary
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams

# Check for CUDA availability (for information only)
# Note: NLTK's n-gram models are CPU-based and don't use GPU acceleration
# They use simple statistical counting which is fast on CPU
try:
    import torch
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        print(f"ℹ️  CUDA is available (GPU: {torch.cuda.get_device_name(0)})")
        print("   Note: NLTK n-gram models run on CPU (GPU not used for this model type)")
    else:
        print("ℹ️  CUDA not available - using CPU (normal for n-gram models)")
except ImportError:
    print("ℹ️  PyTorch not installed - using CPU (normal for n-gram models)")

# Download required NLTK resources
# Punkt tokenizer is used for sentence tokenization (splitting text into sentences)
try:
    nltk.data.find('tokenizers/punkt')
    print("✓ Punkt tokenizer already available")
except LookupError:
    print("Downloading punkt tokenizer...")
    nltk.download('punkt', quiet=True)
    print("✓ Punkt tokenizer downloaded")

# Punkt_tab is an alternative tokenizer that may be needed
try:
    nltk.data.find('tokenizers/punkt_tab')
    print("✓ Punkt_tab tokenizer already available")
except LookupError:
    print("Downloading punkt_tab tokenizer...")
    nltk.download('punkt_tab', quiet=True)
    print("✓ Punkt_tab tokenizer downloaded")

print("\n✅ All libraries and resources ready!")

ℹ️  CUDA is available (GPU: Tesla T4)
   Note: NLTK n-gram models run on CPU (GPU not used for this model type)
✓ Punkt tokenizer downloaded
✓ Punkt_tab tokenizer downloaded

✅ All libraries and resources ready!


In [9]:
from huggingface_hub import login
login()


In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

# ჩავტვირთოთ (მაგალითად common_voice, სადაც ტექსტი 'sentence' სვეტშია)
dataset = load_dataset("RichNachos/georgian-corpus", split="train", streaming=True)
small_subset = list(dataset.take(5))

for i, example in enumerate(small_subset):
    # ავტომატურად ვიპოვოთ ტექსტური სვეტი (ხშირად არის 'text', 'sentence' ან 'content')
    text_key = 'text' if 'text' in example else list(example.keys())[0]

    content = example[text_key]
    print(f"ჩანაწერი {i+1} ({text_key}): {str(content)[:150]}...")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/121 [00:00<?, ?it/s]

ჩანაწერი 1 (doc_content): კორონაცია: | 22 თებერვალი 1540 |
მმართ. დასაწყისი: | 9 მაისი 1538 |
მმართ. დასასრული: | 14 დეკემბერი 1542 |
წინამორბედი: | მადლენა ვალუა |
მემკვიდრე: ...
ჩანაწერი 2 (doc_content): სამსახურებრივი მოვალეობისას დაშავებული კეთილმოწყობის სამსახურის თანამშრომელი გარდაიცვალა
პენიტენციურ დაწესებულებაში სასტიკი მოპრობის მსხვერპლს სახელმწ...
ჩანაწერი 3 (doc_content): რუსეთში ძალაუფლების მთელი ისტორიის მანძილზე ეკატერინა ზაიტევას მესამე ათლეტი გახდა, რომელმაც შეძლო 200 კგ წონით მოხსნას, რისთვისაც მან რუსეთში ყველაზე...
ჩანაწერი 4 (doc_content): თბილისი, 14 მარტი - Sputnik. თბილისის საქალაქო სასამართლო 52 წლის ქართველი მილიარდერის ბადრი პატარკაციშვილის მკვლელობის მომზადების შესახებ საქმის არსე...
ჩანაწერი 5 (doc_content): 1912 წლის 17 მარტს ქ. შ. წ. კ. გ. საზოგადოების მთავარ გამგეობას თელავის განყოფილების გამგეობამ გაუგზავნა განყოფილების 1912 წლის შემოსავალ-გასავლის, გა...


In [4]:
dataset

IterableDataset({
    features: ['doc_content', 'fetch_time', 'url', 'warc_filename', 'warc_record_length', 'warc_record_offset'],
    num_shards: 120
})

In [11]:
# Download and display corpus
texts = []
for i, example in enumerate(dataset):
    if i >= 1000:
        break

    # Automatically find text column (could be 'text', 'sentence', 'content', etc.)
    # First check if 'text' key exists, otherwise use the first available key
    text_key = 'text' if 'text' in example else list(example.keys())[0]
    content = example[text_key]

    # Only add non-empty content
    if content:
        texts.append(str(content))

print(f"✅ Loaded {len(texts)} texts from corpus")

print(f"\n📊 Corpus Statistics:")
print(f"   Total texts: {len(texts)}")
print(f"\n📝 Sample text (first 200 characters):")
if texts:
    print(f"   {texts[0][:200]}{'...' if len(texts[0]) > 200 else ''}")
else:
    print("   No texts loaded!")

✅ Loaded 1000 texts from corpus

📊 Corpus Statistics:
   Total texts: 1000

📝 Sample text (first 200 characters):
   კორონაცია: | 22 თებერვალი 1540 |
მმართ. დასაწყისი: | 9 მაისი 1538 |
მმართ. დასასრული: | 14 დეკემბერი 1542 |
წინამორბედი: | მადლენა ვალუა |
მემკვიდრე: | ფრანსუა დე ვალუა |
დაბ. თარიღი: | 22 ნოემბერი 15...


In [26]:
def tokenize_georgian_text(text):
    """
    Custom tokenizer for Georgian text.

    NLTK's word_tokenize() doesn't handle Georgian script well and splits
    words into individual characters. This function uses a simpler approach:
    split by whitespace and punctuation, but keep Georgian words intact.

    Args:
        text: Input text string

    Returns:
        list: List of word tokens
    """
    import re

    # Convert to lowercase
    text = text.lower()

    # Split by whitespace first
    words = text.split()

    tokens = []
    for word in words:
        # Remove punctuation from start and end, but keep the word
        # This regex keeps Georgian letters, numbers, and common punctuation within words
        # Split on punctuation but keep words together
        word_clean = re.sub(r'^[^\w\u10A0-\u10FF]+|[^\w\u10A0-\u10FF]+$', '', word)

        if word_clean:
            # Check if it contains at least one letter (Georgian or other)
            if any(c.isalpha() for c in word_clean):
                # Only keep tokens with at least 2 characters (filters out single chars)
                # or if it's a number
                if len(word_clean) >= 2 or word_clean.isdigit():
                    tokens.append(word_clean)

    return tokens


def preprocess_text(texts):
    """
    Preprocesses text: tokenization and sentence splitting.

    This function converts raw text into a format suitable for n-gram modeling:
    1. Splits text into sentences
    2. Tokenizes sentences into words using custom Georgian tokenizer
    3. Normalizes text (lowercase)
    4. Filters out single characters and punctuation

    Args:
        texts: List of raw text strings

    Returns:
        list: List of tokenized sentences (each sentence is a list of word tokens)
    """
    print("🔄 Preprocessing text...")
    all_sentences = []

    for text in texts:
        # Split text into sentences using NLTK's sentence tokenizer
        # This handles punctuation, abbreviations, etc. properly
        sentences = sent_tokenize(text)

        for sentence in sentences:
            # Use custom Georgian tokenizer instead of word_tokenize
            # This preserves Georgian words instead of splitting into characters
            tokens = tokenize_georgian_text(sentence)

            # Only add non-empty sentences
            if len(tokens) > 0:
                all_sentences.append(tokens)

    print(f"✅ Processed {len(all_sentences)} sentences")
    return all_sentences

# Preprocess the corpus
sentences = preprocess_text(texts)

# Display example of preprocessed sentence
print(f"\n📝 Example of preprocessed sentence:")
if sentences:
    print(f"   Original: {texts[0][:100]}...")
    print(f"   Tokenized: {sentences[0]}")
    print(f"   (Total tokens in this sentence: {len(sentences[0])})")
else:
    print("   No sentences processed!")

🔄 Preprocessing text...
✅ Processed 25007 sentences

📝 Example of preprocessed sentence:
   Original: კორონაცია: | 22 თებერვალი 1540 |
მმართ. დასაწყისი: | 9 მაისი 1538 |
მმართ. დასასრული: | 14 დეკემბერი...
   Tokenized: ['კორონაცია', 'თებერვალი', 'მმართ']
   (Total tokens in this sentence: 3)


In [27]:
class NGramModel:
    """
    N-gram Language Model Class

    This class implements an n-gram language model using NLTK's MLE (Maximum Likelihood Estimation).
    It can be trained on tokenized sentences and then generate new text based on learned patterns.

    Attributes:
        n: The n-gram order (e.g., 2 for bigram, 3 for trigram)
        model: The trained MLE model from NLTK
        vocab: Vocabulary (set of all unique words in training data)
        train_data: Processed training data
    """

    def __init__(self, n=2):
        """
        Initialize n-gram model with specified order n.

        Args:
            n: Order of n-gram (2=bigram, 3=trigram, etc.)
        """
        self.n = n
        self.model = None
        self.vocab = None
        self.train_data = None

    def train(self, sentences):
        """
        Train the n-gram model on tokenized sentences.

        Process:
        1. Creates padded n-grams (adds start/end markers)
        2. Builds vocabulary from all unique words
        3. Trains MLE model to estimate probabilities

        Args:
            sentences: List of tokenized sentences (each sentence is a list of words)
        """
        print(f"🔄 Training {self.n}-gram model...")

        # Create padded n-grams pipeline
        # This automatically:
        # - Adds <s> (start) and </s> (end) markers to sentences
        # - Creates all n-grams of order 1 to n
        # - Returns training data and vocabulary
        train_data, vocab = padded_everygram_pipeline(self.n, sentences)

        # Create Maximum Likelihood Estimation model
        # MLE estimates probabilities by counting: P(word|context) = count(context+word) / count(context)
        self.model = MLE(self.n)
        self.model.fit(train_data, vocab)

        # Extract vocabulary directly from sentences
        # This is the most reliable method: collect all unique words from tokenized sentences
        # The vocab parameter from padded_everygram_pipeline is a Vocabulary object that's
        # not easily converted, so we extract directly from the source data
        self.vocab = set(word for sentence in sentences for word in sentence)

        # Store training data
        self.train_data = list(train_data)

        print(f"✅ Vocabulary: {len(self.vocab)} unique words")

    def generate_text(self, num_words=20, text_seed=None):
        """
        Generate text using the trained model.

        Process:
        1. Start with seed text (or random words)
        2. For each new word:
           - Use last (n-1) words as context
           - Predict next word using trained probabilities
           - Add to generated text
        3. Repeat until desired length

        Args:
            num_words: Number of words to generate
            text_seed: Optional starting text (seed) for generation

        Returns:
            str: Generated text as a space-separated string
        """
        if self.model is None:
            raise ValueError("Model has not been trained! Call train() first.")

        generated = []
        vocab_list = list(self.vocab)

        if not vocab_list:
            raise ValueError("Vocabulary is empty!")

        # Initialize with seed or random words
        if text_seed:
            # If seed provided, tokenize and use it as starting context
            # Use simple split approach (same logic as tokenize_georgian_text)
            import re
            text_lower = text_seed.lower()
            words = text_lower.split()
            seed_tokens = []
            for word in words:
                word_clean = re.sub(r'^[^\w\u10A0-\u10FF]+|[^\w\u10A0-\u10FF]+$', '', word)
                if word_clean and any(c.isalpha() for c in word_clean):
                    if len(word_clean) >= 2 or word_clean.isdigit():
                        seed_tokens.append(word_clean)
            # Filter: only keep tokens that are in vocabulary
            seed_tokens = [token for token in seed_tokens if token in self.vocab]
            if seed_tokens:
                # Use up to (n-1) tokens as context
                generated = seed_tokens[:self.n-1] if len(seed_tokens) >= self.n-1 else seed_tokens
            else:
                # If seed words not in vocab, start randomly
                generated = [random.choice(vocab_list) for _ in range(min(self.n-1, len(vocab_list)))]
        else:
            # No seed: start with random words
            generated = [random.choice(vocab_list) for _ in range(min(self.n-1, len(vocab_list)))]

        # Generate words one by one
        for _ in range(num_words):
            # Create context: last (n-1) words (or all if fewer available)
            if len(generated) >= self.n - 1:
                context = list(generated[-(self.n-1):])
            else:
                context = list(generated)

            try:
                # Generate next word using NLTK's generate method
                # Returns a list of generated words
                next_words = self.model.generate(1, text_seed=context)
                if next_words and len(next_words) > 0:
                    next_word = next_words[0]
                    # Validate: word must be in vocabulary
                    if next_word and next_word in self.vocab:
                        generated.append(next_word)
                    else:
                        # Fallback: choose random word
                        generated.append(random.choice(vocab_list))
                else:
                    # Generation failed: choose random word
                    generated.append(random.choice(vocab_list))
            except (ValueError, KeyError, IndexError) as e:
                # Error (e.g., context not in vocabulary): choose random word
                generated.append(random.choice(vocab_list))
            except Exception as e:
                # Other errors: choose random word
                generated.append(random.choice(vocab_list))

        return ' '.join(generated)

print("✅ NGramModel class created!")

✅ NGramModel class created!


In [28]:
# Create and train models with different n values
# n=2: Bigram (uses 1 previous word)
# n=3: Trigram (uses 2 previous words)
# n=4: 4-gram (uses 3 previous words)

# First, check if we have the necessary variables
print("="*70)
print("CHECKING PREREQUISITES")
print("="*70)

if 'texts' not in globals() or not texts:
    print("❌ ERROR: 'texts' variable not found or empty!")
    print("   Please run Cell 4 (Download corpus) first.")
    raise NameError("'texts' variable not defined. Run corpus loading cell first.")

if 'sentences' not in globals() or not sentences:
    print("❌ ERROR: 'sentences' variable not found or empty!")
    print("   Please run Cell 6 (Preprocess text) first.")
    print("   Running preprocessing now...")
    sentences = preprocess_text(texts)
    print(f"✅ Preprocessed {len(sentences)} sentences")

print(f"\n✅ Prerequisites check passed:")
print(f"   - Texts loaded: {len(texts) if 'texts' in globals() else 0}")
print(f"   - Sentences processed: {len(sentences) if 'sentences' in globals() else 0}")
if sentences:
    print(f"   - Sample sentence: {sentences[0][:10]}... (first 10 tokens)")
    print(f"   - Total tokens in first sentence: {len(sentences[0])}")

print("\n" + "="*70)
print("TRAINING MODELS WITH DIFFERENT N VALUES")
print("="*70)

n_values = [2, 3, 4]
models = {}

for n in n_values:
    print(f"\n{'='*70}")
    print(f"Training {n}-GRAM model")
    print(f"{'='*70}")

    # Create model instance
    model = NGramModel(n=n)

    # Train on preprocessed sentences
    model.train(sentences)

    # Store trained model
    models[n] = model

    print(f"✅ {n}-gram model training completed!")

print(f"\n✅ All {len(models)} models trained successfully!")

CHECKING PREREQUISITES

✅ Prerequisites check passed:
   - Texts loaded: 1000
   - Sentences processed: 25007
   - Sample sentence: ['კორონაცია', 'თებერვალი', 'მმართ']... (first 10 tokens)
   - Total tokens in first sentence: 3

TRAINING MODELS WITH DIFFERENT N VALUES

Training 2-GRAM model
🔄 Training 2-gram model...
✅ Vocabulary: 78706 unique words
✅ 2-gram model training completed!

Training 3-GRAM model
🔄 Training 3-gram model...
✅ Vocabulary: 78706 unique words
✅ 3-gram model training completed!

Training 4-GRAM model
🔄 Training 4-gram model...
✅ Vocabulary: 78706 unique words
✅ 4-gram model training completed!

✅ All 3 models trained successfully!


In [29]:
print("="*70)
print("GENERATED TEXTS COMPARISON")
print("="*70)
print("\nGenerating text from each model using the same seed for fair comparison...")

# Use a Georgian word as seed to start generation
# This ensures all models start from the same context
seed = "საქართველო"

for n in n_values:
    print(f"\n{'='*70}")
    print(f"{n}-GRAM MODEL GENERATED TEXT")
    print(f"Seed: '{seed}'")
    print(f"{'='*70}")

    try:
        # Generate 30 words starting from the seed
        generated = models[n].generate_text(num_words=30, text_seed=seed)
        print(f"\n{generated}\n")
    except Exception as e:
        print(f"❌ Error during generation: {e}")
        # Try without seed if seed causes issues
        try:
            print("Trying without seed...")
            generated = models[n].generate_text(num_words=30)
            print(f"\n{generated}\n")
        except Exception as e2:
            print(f"❌ Error: {e2}")

print("\n" + "="*70)
print("COMPARISON SUMMARY")
print("="*70)
print("""
Observe the differences:
- 2-gram: Simpler patterns, may be less coherent
- 3-gram: Better context awareness, more natural flow
- 4-gram: Stronger context, but may overfit to training patterns
""")

GENERATED TEXTS COMPARISON

Generating text from each model using the same seed for fair comparison...

2-GRAM MODEL GENERATED TEXT
Seed: 'საქართველო'

საქართველო უამისოდ რატიფიცირების გატარების შეგვაბას მწვრთნელის შეიზღუდება მიღწევებზე ახალგაზრდობასთან ვინერვიულე შემოიღო გაათავისუფლოს სამომავლოს უსაფრთხოების რაბათი აღორძინება მოქმედებები ავტორებთან დაუბრუნდებაო ჩუენდა თქმულის ბოსტან-ქალაქი გადაშალეთ ნარკოტიკები 500-მეტრიანი ფლავონოიდებით დაჰპირდნენ შეხვერდა 15%-მა პარტიაში ლიუისის


3-GRAM MODEL GENERATED TEXT
Seed: 'საქართველო'

საქართველო ერუდიციის დაიტირა ჩიკაგოს მესვეურებს ტყიბულელმა სალიცენზიო სათნო შავრაზმელი ხეთური ჭაბუკებისთვის ბარსელონა-რეალის ინარჩუნებდა გადასახადებთან დამამხობელ შემოფარგლა წამეკითხა მოსლვისაგან ფუნქციის სული ღირებულებებია ხიდ დაიპყრეს ოსტატი ობიექტი პლიუსის დახევის აკუმულატორი ვაერთებ უსიამოვნო პარფიუმერიის


4-GRAM MODEL GENERATED TEXT
Seed: 'საქართველო'

საქართველო წვიმიანი მარიამს მეგობრებს ასოციაციები მივედით ხელაძემ განახლებულ პროფესორ-აკადემიკოსები რა

In [30]:
print("="*70)
print("ADDITIONAL ANALYSIS")
print("="*70)

for n in n_values:
    model = models[n]
    print(f"\n{'─'*70}")
    print(f"{n}-GRAM MODEL STATISTICS")
    print(f"{'─'*70}")
    print(f"  📊 Vocabulary size: {len(model.vocab)} unique words")
    print(f"  🔢 N value: {n} (uses {n-1} previous words for prediction)")

    # Generate multiple examples to show variability
    print(f"\n  📝 Generated examples (showing variability):")
    for i in range(3):
        try:
            text = model.generate_text(num_words=15)
            print(f"     {i+1}. {text}")
        except Exception as e:
            print(f"     {i+1}. ❌ Error: {e}")

print(f"\n{'='*70}")
print("ANALYSIS COMPLETE")
print("="*70)

ADDITIONAL ANALYSIS

──────────────────────────────────────────────────────────────────────
2-GRAM MODEL STATISTICS
──────────────────────────────────────────────────────────────────────
  📊 Vocabulary size: 78706 unique words
  🔢 N value: 2 (uses 1 previous words for prediction)

  📝 Generated examples (showing variability):
     1. შესამჩნევი მკითხველიცაა ბუდობს ხელფასში ლორისს მხატვრებზე გამომდინარეა მარკას გონი უმდიდრესი დაყენებისას შევიწროების ტანსაცმლით დამიკავშირდეთ საშვალებას პრემიერ-მინისტრმა
     2. გადაიქცა დამარცხდა დაბრკოლებულიყო movie.ge-ზე მიენიჭოთ ტრანსსასაზღვრო აღმოუჩენიათ თვალსაზრისით დაჯდება დონორების გადაღებულ მლეთასთან ღამისა პრაქტიკოს გადაიწერა დავიცავთ
     3. შებრძანდეთ დააფინანსებს რძესავით მთაზე ზონიდან არომათერაპია ზოგადზნეობრივი დავთესეთ გარედან კალია კალაპოტს კოხტა დაახლოებული დავასთან საზოგადოებრივობის ოძრხოსის

──────────────────────────────────────────────────────────────────────
3-GRAM MODEL STATISTICS
───────────────────────────────────────────────────